#### PREPARATION

In [28]:
# Reference: https://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script/15622021#15622021
import sys
sys.path.append(r'S:\\')

In [29]:
import os

import numpy as np
import pandas as pd

import rasterio as rio
import xarray as xr
import rioxarray as rxr

import geopandas as gpd

from rasterio.features import shapes

from sklearn.metrics import mean_squared_error

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_style("ticks") # Ref: https://seaborn.pydata.org/tutorial/aesthetics.html

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # For creating grid spec

In [30]:
main_dir = r"S:\Bathymetry\versions010\validation\neal"

-------------------------------------

#### VALIDATION

#### 1. Get functions

In [31]:
# Ref: https://gis.stackexchange.com/questions/317391/python-extract-raster-values-at-point-locations
def point_raster_join(pts_df, path):
    '''A function is to get values from raster at the points'''
    coords = [(x, y) for x, y in zip(pts_df.x, pts_df.y)]
    mx_depth = rio.open(path)
    pts_list = [x[0] for x in mx_depth.sample(coords)]
    return pts_list

In [32]:
def get_dict(value_name, iterative_range, filename, observed_df):
    '''A function is to get values from multiple rasters at the points'''
    calibration_dict = {}
    # Looping to get data
    for i in range(len(iterative_range)):
        # Get dataframe
        path = fr"{main_dir}\\n_{n_calibration[i]}\\{filename}"
        calibration_df = observed_df.copy(deep=True)
        calibration_df[f'{value_name}'] = point_raster_join(calibration_df, path)
        calibration_df[f'{value_name}'] = calibration_df[f'{value_name}'].replace(-9999, np.nan)
        calibration_dict[f"n_{n_calibration[i]}"] = calibration_df[['level', f"{value_name}"]]
    return calibration_dict

#### 2. Get observed data

In [33]:
# Get observed data
obs_data_df = gpd.read_file(fr"{main_dir}\2005b_Flood.shp")
# Choose geometry and level
debris_df = obs_data_df[['geometry', 'X', 'Y', 'level_']]
# Rename
debris_df.rename(columns={'X':'x', 'Y':'y', 'level_':'level'}, inplace=True)
# Copy the dataframe and call it validation dataframe
validation_df = debris_df.copy(deep=True)

C:\Users\tmn52\AppData\Local\Temp\ipykernel_26600\4029149510.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  debris_df.rename(columns={'X':'x', 'Y':'y', 'level_':'level'}, inplace=True)


In [34]:
obs_data_df.shape

(32, 18)

In [35]:
validation_df

,geometry,x,y,level
0,POINT (1769595.050 5472746.640),1769595.05,5472746.64,2.203
1,POINT (1769703.530 5472851.610),1769703.53,5472851.61,2.559
2,POINT (1769770.480 5473021.640),1769770.48,5473021.64,2.677
3,POINT (1769902.520 5472848.810),1769902.52,5472848.81,3.124
4,POINT (1769420.780 5473245.410),1769420.78,5473245.41,1.635
5,POINT (1770036.680 5472870.030),1770036.68,5472870.03,3.800
6,POINT (1770093.230 5472756.470),1770093.23,5472756.47,3.792
7,POINT (1770191.100 5472671.230),1770191.10,5472671.23,4.424
8,POINT (1770358.080 5472791.900),1770358.08,5472791.90,4.586
9,POINT (1770891.560 5472949.040),1770891.56,5472949.04,5.014


In [36]:
# VERSION NOT COUNTING FOR ERRORS FROM DATA COLLECTION
obs_data_df_copy = obs_data_df.drop(index=[
    20,
    2, 6, 7, 11, 15, 17, 25, 26, 27 # SAME errors from python
])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005t_Flood.shp", crs=2193)

In [27]:
# VERSION NOT COUNTING FOR ERRORS FROM DATA COLLECTION
obs_data_df_copy = obs_data_df.drop(index=[
    12, 13, 20, 24,
    2, 6, 7, 11, 15, 17, 25, 26, 27 # SAME errors from python
])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005s_Flood.shp", crs=2193)

In [18]:
# VERSION NOT COUNTING FOR ERRORS FROM DATA COLLECTION
obs_data_df_copy = obs_data_df.drop(index=[
    2, 6, 7, 11, 15, 17, 25, 26, 27 # SAME errors from python
])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005r_Flood.shp", crs=2193)

In [9]:
# VERSION NOT COUNTING FOR ERRORS FROM DATA COLLECTION
obs_data_df_copy = obs_data_df.drop(index=[
    2, 4, 5, 6, 7, 11, 15, 16, 17, 26, 25, 27, # SAME errors from python
    20, 30, 31 # outliers errors from python
])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005q_Flood.shp", crs=2193)

In [ ]:
obs_data_df_copy = obs_data_df.drop(index=[
    2, 4, 5, 6, 7, 11, 15, 16, 17, 25, 27, # SAME errors from python
    30, 31 # outliers errors from python
])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005o_Flood.shp", crs=2193)

In [ ]:
# VERSION COUNTING FOR ERRORS FROM DATA COLLECTION & FROM PYTHON
obs_data_df_copy = obs_data_df.drop(index=[
    11, 12, 13, 20, 24, 25, 26, 27, # errors from data collection
    2, 4, 5, 6, 7, 15, 16, 17, 30, 31 # errors from python
])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005n_Flood.shp", crs=2193)

In [ ]:
# Choose geometry and level
debris_df = obs_data_df_copy[['geometry', 'X', 'Y', 'level_']]
# Rename
debris_df.rename(columns={'X':'x', 'Y':'y', 'level_':'level'}, inplace=True)
# Copy the dataframe and call it validation dataframe
validation_df = debris_df.copy(deep=True)

In [ ]:
validation_df

In [ ]:
obs_data_df_copy = obs_data_df.drop(index=[2, 7, 12, 13, 11, 15, 17, 20, 24, 25, 26, 27, 30, 31])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005k_Flood.shp", crs=2193)

In [ ]:
obs_data_df_copy = obs_data_df.drop(index=[2, 7, 12, 13, 15, 17, 19, 20, 24, 25, 26, 27, 30, 31])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005g_Flood.shp", crs=2193)

In [ ]:
## The best one
obs_data_df_copy = obs_data_df.drop(index=[2, 7, 12, 13, 15, 17, 19, 20, 24, 25, 26, 27, 30, 31])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005g_Flood.shp", crs=2193)

In [ ]:
obs_data_df_copy = obs_data_df.drop(index=[2, 7, 12, 13, 15, 17, 19, 20, 24, 25, 26, 27])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005f_Flood.shp", crs=2193)

In [ ]:
obs_data_df_copy = obs_data_df.drop(index=[2, 7, 12, 13, 17, 19, 20, 24, 25, 26, 27])
obs_data_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005d_Flood.shp", crs=2193)

In [ ]:
debris_df_copy = debris_df.drop(index=[12, 13, 20, 24, 25, 26, 27])

In [ ]:
debris_df_copy.to_file(fr"S:\Bathymetry\versions010\validation\2005a_Flood.shp", crs=2193)

In [ ]:
%%time
# Get level from model results
validation_df['mxe'] = point_raster_join(debris_df, fr"{main_dir}\neal_flood_result.nc")

#### 3. Calculate errors

In [ ]:
# Calculate the error
validation_df_copy = validation_df.copy(deep=True)
validation_df_copy['error'] = validation_df['mxe'] - validation_df_copy['level']

In [ ]:
validation_df_copy_filter = validation_df_copy.loc[
    validation_df_copy['error'] >= -5, :
]

In [ ]:
# Get avarge error and avarage absolute error
print(validation_df_copy_filter['error'].mean())
print(validation_df_copy_filter['error'].abs().mean())

#### 4. Get plot

In [ ]:
# Validate with mxe and rmse
validation_mxe_mse = mean_squared_error(validation_df.level, validation_df.mxe, squared=True)
validation_mxe_rmse = mean_squared_error(validation_df.level, validation_df.mxe, squared=False)

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(7, 7))

# Size for title and label
fontsize = 14
labelpad = 21

# Plot
sns.regplot(x='level', y='mxe', data=validation_df,
            scatter_kws={"s": 50, 'edgecolor': 'black', 'color':'deeppink', 'linewidth':.7},
            line_kws={'color':'darkred', 'linewidth':1.5}, marker='o', ci=95, ax=ax)

# Adjust x and y labels
ax.set_xlabel("Observed data - Level (m)", fontsize=fontsize, labelpad=labelpad)
ax.set_ylabel("Predicted data -\nmaximum water surface elevation (m)", rotation=-270, fontsize=fontsize, labelpad=labelpad+5)

# Set up ticks
ax.set_yticks(np.arange(2.5, 21, 2.5))

# For frame
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    
# Set up ticks
for item in (ax.get_xticklabels() + ax.get_yticklabels()):  # For x, y ticks' labels
    item.set_fontsize(fontsize-3)
ax.tick_params(direction='out', length=5, pad=labelpad-17)
    

title_error = "MSE:\n\nRMSE:"
error = f"{validation_mxe_mse:.3f}\n\n{validation_mxe_rmse:.3f}"

# Error added into the text
# Ref: https://github.com/matplotlib/matplotlib/issues/253/
#      https://stackoverflow.com/questions/67366092/valueerror-alignment-not-allowed-in-string-format-specifier-sometimes-not
#      https://stackoverflow.com/questions/8234445/format-output-string-right-alignment
ax.text(
    .1, .8, # Control the text on the x axis and y axis
    title_error,
    size=fontsize-2, ha='left', color='black', transform=ax.transAxes
)
ax.text(
    .22, .8, # Control the text on the x axis and y axis
    error,
    size=fontsize-2, ha='left', color='black', transform=ax.transAxes
)

plt.savefig(fr"{main_dir}\validation_result.png", bbox_inches='tight', dpi=600)